In [1]:
import pandas as pd
df_products_translated=pd.read_csv('./data/brazilian_e-commerce/olist_products_dataset_translated.csv')
df_products_translated.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,Perfumes
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,Arts
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,Sports and Leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,Baby Products
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,Household Essentials


In [5]:
df_products_translated['product_category_name_english'].value_counts()

product_category_name_english
Bed and Bathroom Products             3029
Sports and Leisure                    2867
Furniture and Decoration              2657
Beauty and Health Products            2444
Household Essentials                  2335
                                      ... 
Home Comfort (2)                         5
Children's and Teenagers' Clothing       5
PC Gamer                                 3
Insurance and Services                   2
Music CDs and DVDs                       1
Name: count, Length: 72, dtype: int64

In [6]:
df_products_translated['product_category_name_english'].unique()

array(['Perfumes', 'Arts', 'Sports and Leisure', 'Baby Products',
       'Household Essentials', 'Musical Instruments', 'Cool Stuff',
       'Furniture and Decoration', 'Electrical Appliances', 'Toys',
       'Bed and Bathroom Products',
       'Construction Tools and Safety Equipment', 'Computer Accessories',
       'Beauty and Health Products', 'Luggage and Travel Accessories',
       'Garden Tools', 'Office Furniture', 'Automotive Products',
       'Electronics', 'Footwear and Fashion', 'Telephony',
       'Paper Products', 'Handbags and Fashion Accessories',
       'Personal Care Products', 'Home Construction',
       'Watches and Gift Items', 'Construction Tools for Construction',
       'Pet Shop', 'Electrical Portables',
       'Agriculture, Industry, and Commerce', nan,
       'Living Room Furniture', 'Signage and Security Equipment',
       'Climate Control', 'Consoles and Games',
       'Books and General Interest Products',
       'Underwear and Beach Fashion', "Men's Clothi

In [11]:
import requests
import json

def ollama_chat(prompt):
    url = "http://localhost:11434/api/chat"
    payload = {
        "model": "llama3:8b",
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(url, json=payload, stream=True)

    full_output = ""

    for line in response.iter_lines():
        if not line:
            continue

        try:
            data = json.loads(line.decode("utf-8"))
        except json.JSONDecodeError:
            print("Skipping invalid line:", line)
            continue

        # Collect streamed content
        if "message" in data and "content" in data["message"]:
            full_output += data["message"]["content"]

    return full_output


In [12]:
categories = df_products_translated['product_category_name_english'].unique().tolist()

prompt = f"""
You are a categorization expert.

Below is a list of product categories ({len(categories)} items):
{categories}

Please:
1. Compress them into a smaller number of broader categories.
2. Each broad category must be logical and semantically consistent.
3. Return ONLY JSON in this format:

{{
  "clusters": [
    {{
      "cluster_name": "string",
      "items": ["item1", "item2", ...]
    }}
  ]
}}
"""

response = ollama_chat(prompt)
print(response)


After analyzing the list, I was able to compress the categories into a smaller number of broader categories that are logical and semantically consistent. Here is the result:

{
  "clusters": [
    {
      "cluster_name": "Home and Living",
      "items": ["Perfumes", "Baby Products", "Household Essentials", "Furniture and Decoration", "Bed and Bathroom Products", "Living Room Furniture", "Kitchenware and Food Preparation Tools", "Kitchen and Dining Room Furniture", "Home Comfort", "Home Comfort (2)", "Bedding and Upholstery"]
    },
    {
      "cluster_name": "Electronics",
      "items": ["Musical Instruments", "Computer Accessories", "Electronics", "Tablets and Image Printing", "PC Gamer", "Audio", "DVDs and Blu-rays", "Music CDs and DVDs", "Consoles and Games"]
    },
    {
      "cluster_name": "Fashion and Clothing",
      "items": ["Footwear and Fashion", "Handbags and Fashion Accessories", "Women's Fashion Clothing", "Men's Clothing", "Children's and Teenagers' Clothing", "Spor

In [14]:
print(response)


After analyzing the list, I was able to compress the categories into a smaller number of broader categories that are logical and semantically consistent. Here is the result:

{
  "clusters": [
    {
      "cluster_name": "Home and Living",
      "items": ["Perfumes", "Baby Products", "Household Essentials", "Furniture and Decoration", "Bed and Bathroom Products", "Living Room Furniture", "Kitchenware and Food Preparation Tools", "Kitchen and Dining Room Furniture", "Home Comfort", "Home Comfort (2)", "Bedding and Upholstery"]
    },
    {
      "cluster_name": "Electronics",
      "items": ["Musical Instruments", "Computer Accessories", "Electronics", "Tablets and Image Printing", "PC Gamer", "Audio", "DVDs and Blu-rays", "Music CDs and DVDs", "Consoles and Games"]
    },
    {
      "cluster_name": "Fashion and Clothing",
      "items": ["Footwear and Fashion", "Handbags and Fashion Accessories", "Women's Fashion Clothing", "Men's Clothing", "Children's and Teenagers' Clothing", "Spor

In [15]:
import re
import json

def extract_json(text):
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if not match:
        raise ValueError("No JSON found.")
    return json.loads(match.group(0))


In [16]:
clusters_json = extract_json(response)


In [18]:
mapping = {}

for cluster in clusters_json["clusters"]:
    name = cluster["cluster_name"]
    for item in cluster["items"]:
        mapping[item] = name

mapping

{'Perfumes': 'Beauty and Health',
 'Baby Products': 'Home and Living',
 'Household Essentials': 'Home and Living',
 'Furniture and Decoration': 'Home and Living',
 'Bed and Bathroom Products': 'Home and Living',
 'Living Room Furniture': 'Home and Living',
 'Kitchenware and Food Preparation Tools': 'Home and Living',
 'Kitchen and Dining Room Furniture': 'Home and Living',
 'Home Comfort': 'Home and Living',
 'Home Comfort (2)': 'Home and Living',
 'Bedding and Upholstery': 'Home and Living',
 'Musical Instruments': 'Electronics',
 'Computer Accessories': 'Office and Industry',
 'Electronics': 'Electronics',
 'Tablets and Image Printing': 'Electronics',
 'PC Gamer': 'Electronics',
 'Audio': 'Electronics',
 'DVDs and Blu-rays': 'Electronics',
 'Music CDs and DVDs': 'Electronics',
 'Consoles and Games': 'Leisure and Entertainment',
 'Footwear and Fashion': 'Fashion and Clothing',
 'Handbags and Fashion Accessories': 'Fashion and Clothing',
 "Women's Fashion Clothing": 'Fashion and Clothi

In [20]:
df_products_translated["compressed_category"] = (
    df_products_translated["product_category_name_english"].map(mapping)
)
df_products_translated

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,compressed_category
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,Perfumes,Beauty and Health
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,Arts,Leisure and Entertainment
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,Sports and Leisure,Leisure and Entertainment
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,Baby Products,Home and Living
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,Household Essentials,Home and Living
...,...,...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0,Furniture and Decoration,Home and Living
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0,Building Tools and Lighting,Home Improvement
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0,Bed and Bathroom Products,Home and Living
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0,Computer Accessories,Office and Industry


In [22]:
df_products_translated['compressed_category'].value_counts()

compressed_category
Home and Living              9326
Leisure and Entertainment    4731
Beauty and Health            3354
Office and Industry          2502
Home Improvement             1685
Fashion and Clothing         1221
Electronics                   925
Travel and Luggage            349
Food and Beverage             187
Name: count, dtype: int64

In [23]:
df_products_translated['compressed_category'].unique()

array(['Beauty and Health', 'Leisure and Entertainment',
       'Home and Living', 'Electronics', nan, 'Home Improvement',
       'Office and Industry', 'Travel and Luggage',
       'Fashion and Clothing', 'Food and Beverage'], dtype=object)

In [24]:
df_products_translated['compressed_category'].nunique()

9